In [2]:
import os

In [3]:
%pwd

'd:\\project\\Predict-Lung-Disease\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\project\\Predict-Lung-Disease'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=config.root_dir,
            tensorboard_root_log_dir=config.tensorboard_root_log_dir,
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callbacks_config

In [10]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter
import time

In [13]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )

        self.writer = SummaryWriter(log_dir=tb_running_log_dir)
        return self.writer
    
    def _create_ckpt_callbacks(self, model, optimizer, epoch):
        checkpoint_path = self.config.checkpoint_model_filepath

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': self.config.loss 
        }, checkpoint_path)

    def get_tb_ckpt_callbacks(self, model=None, optimizer=None, epoch=None):
        return {
            "tensorboard": self._create_tb_callbacks,
            "checkpoint": lambda: self._create_ckpt_callbacks(model, optimizer, epoch)
        }

In [19]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-10-07 14:45:23,041: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-07 14:45:23,047: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-07 14:45:23,049: INFO: common: creating directory at: artifacts]
[2024-10-07 14:45:23,079: INFO: common: creating directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-07 14:45:23,081: INFO: common: creating directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
